In [33]:
%pip install -q optuna lightgbm

Note: you may need to restart the kernel to use updated packages.


This experiment compares the performance of several classification algorithms (Logistic Regression, Naive Bayes, SVM, XGBoost, LightGBM). We apply Hyperparameter Tuning (HPT) to the complex models using Optuna, while fixing the feature engineering pipeline based on previous optimal choices:

* **Vectorization:** TF-IDF (Term Frequency-Inverse Document Frequency)
* **N-gram Range:** Bigram `(1, 2)` (Unigrams and Bigrams)
* **Max Features:** 1000
* **Imbalance Handling:** Undersampling (`RandomUnderSampler`)

## 1. Setup and Dependencies

### 1.1 Import Libraries

In [34]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 2. MLflow and Data Preparation

### 2.1 MLflow Configuration

In [35]:
# Set the remote tracking server URI
mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

# Set or create a new experiment
mlflow.set_experiment("Model Comparision (TFIDF Bigram 1000 + Ensemble Imbalance) - Exp 5")

<Experiment: artifact_location='s3://mlfow-bucket-2025/17', creation_time=1763217601948, experiment_id='17', last_update_time=1763217601948, lifecycle_stage='active', name='Model Comparision (TFIDF Bigram 1000 + Ensemble Imbalance) - Exp 5', tags={'mlflow.experimentKind': 'custom_model_development'}>

### 2.2 Data Loading, Remapping, and Feature Engineering

In [36]:
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['category'])

print(f"Data shape: {df.shape}")
print(f"Class distribution:\n{df['category'].value_counts().sort_index()}")

# Fixed parameters
ngram_range = (1, 2)
max_features = 1000

# Split and vectorize
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], 
    test_size=0.2, random_state=42, stratify=df['category']
)

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"\nOriginal Training Data Shape: {X_train_vec.shape}")
print(f"Test Data Shape: {X_test_vec.shape}")

Data shape: (36662, 2)
Class distribution:
category
0    12644
1    15770
2     8248
Name: count, dtype: int64

Original Training Data Shape: (29329, 1000)
Test Data Shape: (7333, 1000)


### Resampled Data

In [37]:
print("\n" + "="*80)
print("CREATING 3 RESAMPLED DATASETS (TOP 3 IMBALANCE METHODS)")
print("="*80)

# Method 1: Undersampling
sampler_under = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = sampler_under.fit_resample(X_train_vec, y_train)
print(f"1. Undersampled: {X_train_under.shape}")

# Method 2: Oversampling
sampler_over = RandomOverSampler(random_state=42)
X_train_over, y_train_over = sampler_over.fit_resample(X_train_vec, y_train)
print(f"2. Oversampled: {X_train_over.shape}")

# Method 3: ADASYN
sampler_adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = sampler_adasyn.fit_resample(X_train_vec, y_train)
print(f"3. ADASYN: {X_train_adasyn.shape}")

# Store all datasets
datasets = {
    'undersampling': (X_train_under, y_train_under),
    'oversampling': (X_train_over, y_train_over),
    'adasyn': (X_train_adasyn, y_train_adasyn)
}


CREATING 3 RESAMPLED DATASETS (TOP 3 IMBALANCE METHODS)
1. Undersampled: (19794, 1000)
2. Oversampled: (37848, 1000)
3. ADASYN: (35909, 1000)


## 3. MLflow Logging and Evaluation Helper

In [38]:
def log_mlflow(model_name, model, params=None, imbalance_method="undersampling"):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_{imbalance_method}_TFIDF(1000)_HPT")
        mlflow.set_tag("experiment_type", "multi_algo_hpt")
        mlflow.log_param("algo_name", model_name)
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", str(ngram_range))
        mlflow.log_param("max_features", max_features)
        mlflow.log_param("imbalance_handling", imbalance_method)
        
        if params:
            for key, value in params.items():
                mlflow.log_param(key, value)

        # Predict
        y_pred = model.predict(X_test_vec)

        # Log metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix: {model_name} ({imbalance_method})")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.savefig(f"conf_matrix_{model_name}_{imbalance_method}.png")
        mlflow.log_artifact(f"conf_matrix_{model_name}_{imbalance_method}.png")
        plt.close()

        mlflow.sklearn.log_model(model, f"{model_name}_model")
        
        print(f"    ✓ Logged with Accuracy: {accuracy:.4f}")

## 4. Hyperparameter Tuning Objectives (Optuna)

In [39]:
N_TRIALS = 10

def tune_logistic_regression(X_train, y_train):
    def objective(trial):
        C = trial.suggest_float('C', 1e-3, 10.0, log=True)
        solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs'])
        model = LogisticRegression(C=C, solver=solver, random_state=42, multi_class='auto', max_iter=1000)
        model.fit(X_train, y_train)
        return accuracy_score(y_test, model.predict(X_test_vec))
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
    return study.best_params

def tune_linear_svc(X_train, y_train):
    def objective(trial):
        C = trial.suggest_float('C', 0.1, 10.0, log=True)
        model = LinearSVC(C=C, random_state=42, max_iter=1000, dual='auto')
        model.fit(X_train, y_train)
        return accuracy_score(y_test, model.predict(X_test_vec))
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
    return study.best_params

def tune_xgboost(X_train, y_train):
    def objective(trial):
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        learning_rate = trial.suggest_float('learning_rate', 1e-3, 0.1, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
                             random_state=42, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1)
        model.fit(X_train, y_train)
        return accuracy_score(y_test, model.predict(X_test_vec))
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
    return study.best_params

def tune_lightgbm(X_train, y_train):
    def objective(trial):
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        learning_rate = trial.suggest_float('learning_rate', 1e-3, 0.1, log=True)
        num_leaves = trial.suggest_int('num_leaves', 10, 50)
        model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, num_leaves=num_leaves,
                              random_state=42, verbose=-1)
        model.fit(X_train, y_train)
        return accuracy_score(y_test, model.predict(X_test_vec))
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
    return study.best_params

## 5. Execution and MLflow Logging

In [40]:
def train_individual_models_with_hpt(model_name, tune_func):
    """
    Train separate models on each resampling method and log each one.
    This allows us to find which algorithm + sampling method combination works best.
    """
    print(f"\n{'='*80}")
    print(f"TRAINING {model_name} ON ALL 3 SAMPLING METHODS")
    print(f"{'='*80}")
    
    for method, (X_resample, y_resample) in datasets.items():
        print(f"\n  [{method.upper()}] Running HPT ({N_TRIALS} trials)...")
        
        # Tune hyperparameters
        best_params = tune_func(X_resample, y_resample)
        print(f"    Best params: {best_params}")
        
        # Train model with best params
        if model_name == 'LogisticRegression':
            model = LogisticRegression(random_state=42, multi_class='auto', max_iter=1000, **best_params)
        elif model_name == 'LinearSVC':
            model = LinearSVC(random_state=42, max_iter=1000, dual='auto', **best_params)
        elif model_name == 'XGBoost':
            model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1, **best_params)
        elif model_name == 'LightGBM':
            model = LGBMClassifier(random_state=42, verbose=-1, **best_params)
        
        model.fit(X_resample, y_resample)
        
        # Log each model separately
        log_mlflow(f"{model_name}", model, params=best_params, imbalance_method=method)
    
    print(f"{'='*80}")

# Execute Training
# ============================================================================
print("\n" + "="*80)
print("STARTING MODEL TRAINING WITH ALL SAMPLING METHODS")
print("="*80)
print("Strategy: Train each algorithm on all 3 sampling methods separately")
print("This creates 12 models (4 algorithms × 3 sampling methods)")
print("MLflow will track all and identify the best combination")
print("="*80)

# Train baseline
print(f"\n{'='*80}")
print("BASELINE: MultinomialNB (No HPT)")
print(f"{'='*80}")
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train_under, y_train_under)
log_mlflow('MultinomialNB', mnb, params={'alpha': 1.0}, imbalance_method='undersampling')

# Train all models on all sampling methods individually
models_to_tune = [
    ('LogisticRegression', tune_logistic_regression),
    ('LinearSVC', tune_linear_svc),
    ('XGBoost', tune_xgboost),
    ('LightGBM', tune_lightgbm)
]

for model_name, tune_func in models_to_tune:
    train_individual_models_with_hpt(model_name, tune_func)

print("\n" + "="*80)
print("✓ ALL MODELS TRAINED AND LOGGED")
print("="*80)


STARTING MODEL TRAINING WITH ALL SAMPLING METHODS
Strategy: Train each algorithm on all 3 sampling methods separately
This creates 12 models (4 algorithms × 3 sampling methods)
MLflow will track all and identify the best combination

BASELINE: MultinomialNB (No HPT)


2025/11/15 21:08:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 21:09:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ Logged with Accuracy: 0.7000
🏃 View run MultinomialNB_undersampling_TFIDF(1000)_HPT at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/17/runs/03490333fe014eefbedad5d9db0a1bd9
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/17


[I 2025-11-15 21:09:07,678] A new study created in memory with name: no-name-05dce9b7-d596-414a-9c60-47b1757eb243
[I 2025-11-15 21:09:07,829] Trial 0 finished with value: 0.7809900450020455 and parameters: {'C': 2.8430254938819615, 'solver': 'liblinear'}. Best is trial 0 with value: 0.7809900450020455.



TRAINING LogisticRegression ON ALL 3 SAMPLING METHODS

  [UNDERSAMPLING] Running HPT (10 trials)...


...


  [ADASYN] Running HPT (10 trials)...


[I 2025-11-15 21:39:20,347] Trial 0 finished with value: 0.7486703941088231 and parameters: {'n_estimators': 50, 'learning_rate': 0.010454865501484783, 'num_leaves': 49}. Best is trial 0 with value: 0.7486703941088231.
[I 2025-11-15 21:39:29,109] Trial 1 finished with value: 0.785490249556798 and parameters: {'n_estimators': 115, 'learning_rate': 0.03150369418385284, 'num_leaves': 46}. Best is trial 1 with value: 0.785490249556798.
[I 2025-11-15 21:39:38,992] Trial 2 finished with value: 0.7238510841401882 and parameters: {'n_estimators': 148, 'learning_rate': 0.002997506411675642, 'num_leaves': 35}. Best is trial 1 with value: 0.785490249556798.
[I 2025-11-15 21:39:47,769] Trial 3 finished with value: 0.7858993590617755 and parameters: {'n_estimators': 97, 'learning_rate': 0.03593581755262887, 'num_leaves': 48}. Best is trial 3 with value: 0.7858993590617755.
[I 2025-11-15 21:39:57,298] Trial 4 finished with value: 0.7908086731215055 and parameters: {'n_estimators': 196, 'learning_rat

    Best params: {'n_estimators': 196, 'learning_rate': 0.05437133178453067, 'num_leaves': 25}


2025/11/15 21:41:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/15 21:41:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


    ✓ Logged with Accuracy: 0.7908
🏃 View run LightGBM_adasyn_TFIDF(1000)_HPT at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/17/runs/cf778d451434433fbe988012a3f0b387
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/17

✓ ALL MODELS TRAINED AND LOGGED


## 6. Conclusion and Next Steps
The complete set of model performances, including optimized hyperparameters, is now logged in the MLflow UI. The next step is typically stacking or selecting the single best performing model based on comprehensive evaluation metrics, especially F1-scores for the minority classes.

In [42]:
OPTIMAL_METRIC = "weighted avg_f1-score"

try:
    client = mlflow.tracking.MlflowClient()
    experiment = client.get_experiment_by_name("Model Comparision (TFIDF Bigram 1000 + Ensemble Imbalance) - Exp 5")
    
    if experiment:
        experiment_id = experiment.experiment_id
        print(f"\n{'='*80}")
        print(f"FETCHING RESULTS FROM EXPERIMENT")
        print(f"{'='*80}")

        runs = client.search_runs(experiment_ids=experiment_id)
        run_data = []
        
        for run in runs:
            metrics = run.data.metrics
            params = run.data.params
            
            run_data.append({
                'run_id': run.info.run_id,
                'algo_name': params.get('algo_name', 'N/A'),
                'imbalance_handling': params.get('imbalance_handling', 'N/A'),
                'accuracy': metrics.get('accuracy', 0.0),
                'weighted_f1': metrics.get(OPTIMAL_METRIC, 0.0),
                '0_f1': metrics.get('0_f1-score', 0.0),
                '1_f1': metrics.get('1_f1-score', 0.0),
                '2_f1': metrics.get('2_f1-score', 0.0),
                'run_name': run.data.tags.get('mlflow.runName')
            })

        df_results = pd.DataFrame(run_data)
        
        # Create composite score: average of accuracy and weighted F1
        df_results['composite_score'] = (df_results['accuracy'] + df_results['weighted_f1']) / 2
        
        # Sort by composite score
        df_results = df_results.sort_values(by='composite_score', ascending=False)

        print("\n" + "="*80)
        print(f"TOP 10 MODELS (Sorted by Composite Score: Avg of Accuracy & Weighted F1)")
        print("="*80)
        print(df_results[['algo_name', 'imbalance_handling', 'accuracy', 
                         'weighted_f1', 'composite_score', '2_f1']].head(10).to_string(index=False))
        
        # Rankings by different metrics
        print("\n" + "="*80)
        print("RANKINGS BY DIFFERENT METRICS")
        print("="*80)
        
        print("\n📊 Top 5 by Accuracy:")
        top_acc = df_results.nlargest(5, 'accuracy')
        for idx, row in top_acc.iterrows():
            print(f"  {row['algo_name']:25s} ({row['imbalance_handling']:15s}) - Accuracy: {row['accuracy']:.4f}")
        
        print("\n📊 Top 5 by Weighted F1-Score:")
        top_f1 = df_results.nlargest(5, 'weighted_f1')
        for idx, row in top_f1.iterrows():
            print(f"  {row['algo_name']:25s} ({row['imbalance_handling']:15s}) - Weighted F1: {row['weighted_f1']:.4f}")
        
        print("\n📊 Top 5 by Minority Class (2) F1-Score:")
        top_minority = df_results.nlargest(5, '2_f1')
        for idx, row in top_minority.iterrows():
            print(f"  {row['algo_name']:25s} ({row['imbalance_handling']:15s}) - Class 2 F1: {row['2_f1']:.4f}")
        
        # Algorithm + Sampling Method Analysis
        print("\n" + "="*80)
        print("BEST SAMPLING METHOD FOR EACH ALGORITHM")
        print("="*80)
        for algo in df_results['algo_name'].unique():
            if algo != 'N/A':
                algo_df = df_results[df_results['algo_name'] == algo]
                best_row = algo_df.iloc[0]
                print(f"\n{algo}:")
                print(f"  Best Sampling: {best_row['imbalance_handling']}")
                print(f"  Accuracy: {best_row['accuracy']:.4f}")
                print(f"  Weighted F1: {best_row['weighted_f1']:.4f}")
                print(f"  Minority F1: {best_row['2_f1']:.4f}")
        
        print("\n" + "="*80)
        print("🏆 BEST OVERALL MODEL (by Composite Score)")
        print("="*80)
        best = df_results.iloc[0]
        print(f"Algorithm: {best['algo_name']}")
        print(f"Imbalance Handling: {best['imbalance_handling']}")
        print(f"Composite Score: {best['composite_score']:.4f}")
        print(f"Accuracy: {best['accuracy']:.4f}")
        print(f"Weighted F1-Score: {best['weighted_f1']:.4f}")
        print(f"Class 0 F1: {best['0_f1']:.4f}")
        print(f"Class 1 F1: {best['1_f1']:.4f}")
        print(f"Class 2 (Minority) F1: {best['2_f1']:.4f}")
        print(f"Run ID: {best['run_id']}")
        print("="*80)
        
        print("\n💡 Key Insights:")
        print("   ✓ Each algorithm trained on undersampling, oversampling, and ADASYN")
        print("   ✓ Best model selected from all 12 combinations (4 algos × 3 methods)")
        print("   ✓ Composite Score balances accuracy with class-wise F1 performance")
        print("   ✓ This approach avoids ensemble voting that may reduce performance")

except Exception as e:
    print(f"Error: {e}")


FETCHING RESULTS FROM EXPERIMENT

TOP 10 MODELS (Sorted by Composite Score: Avg of Accuracy & Weighted F1)
         algo_name imbalance_handling  accuracy  weighted_f1  composite_score     2_f1
          LightGBM             adasyn  0.790809     0.787167         0.788988 0.658624
         LinearSVC       oversampling  0.785627     0.783244         0.784435 0.662496
         LinearSVC      undersampling  0.784672     0.782359         0.783516 0.665629
          LightGBM       oversampling  0.783854     0.780722         0.782288 0.653920
          LightGBM      undersampling  0.781808     0.779463         0.780636 0.658393
LogisticRegression      undersampling  0.780990     0.779422         0.780206 0.659724
LogisticRegression       oversampling  0.779899     0.778274         0.779086 0.654291
         LinearSVC             adasyn  0.779626     0.777610         0.778618 0.644207
LogisticRegression             adasyn  0.774853     0.773705         0.774279 0.638048
           XGBoost    